In [1]:
import gzip
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import netCDF4 as nc
import match
from netCDF4 import Dataset
import xarray as xr
import os 
import pandas as pd  
from datetime import datetime

# 生成文件目录

In [5]:
start_year = 2011
end_year = 2021

folder_path = 'D:/data/SSW/'  # 文件夹路径
file_names = os.listdir(folder_path)  
len(file_names)  

132

In [6]:
file_new = [] 
for i in range(len(file_names)):
    SSW = xr.open_dataset(folder_path+file_names[i])
    file_new.append(SSW)

In [7]:
len(file_new)

132

In [8]:
data = xr.concat(file_new,dim='time')
data_path = r'D:/data/SSW/2011-2021_concat.nc' 
data.to_netcdf(data_path)

# 读取合并数据

In [9]:
nc = Dataset(data_path)
print(nc.variables.keys())

for var in nc.variables.keys():
    data = nc.variables[var][:].data
    print(var,data.shape)

dict_keys(['u', 'v', 'w', 'n', 'latitude', 'longitude', 'u_anom', 'v_anom', 'w_anom'])
u (132, 720, 1440)
v (132, 720, 1440)
w (132, 720, 1440)
n (132, 720, 1440)
latitude (720,)
longitude (1440,)
u_anom (132, 720, 1440)
v_anom (132, 720, 1440)
w_anom (132, 720, 1440)


In [10]:
nc.variables['u'][:].mask.shape 

(132, 720, 1440)

# 记录数据

In [11]:
lat = []
lon = []
u = []
v = []
time = []

In [12]:
u = nc.variables['u'][:].data
v = nc.variables['v'][:].data

## 获取 lat lon time 

In [14]:
temp = Dataset(folder_path + file_names[0])
lat.append(temp.variables['latitude'][:].data)
lon.append(temp.variables['longitude'][:].data)

In [15]:
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [16]:
len(time)

132

# 保存数据

In [ ]:
time[0]

In [18]:
#new_NC.close()
new_NC = Dataset("D:/data/SSW/2011-2021_SSW_correct.nc", 'w', format='NETCDF4')

new_NC.createDimension('time', len(time))
new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2011-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T' 

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))

new_NC.createVariable('u', 'f', ("time","lat","lon"))
new_NC.createVariable('v', 'f', ("time","lat","lon"))


#向变量中填充数据
new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]

new_NC.variables['time'][:] = np.array(time)

new_NC.variables['u'][:]=np.array(u)
new_NC.variables['v'][:]=np.array(v)

#最后记得关闭文件
new_NC.close()

